In [221]:
import numpy as np
import pandas as pd

df = pd.read_csv('downloaded_dup.csv', low_memory=False)

In [222]:
df.head()

,Id,PayeeId,AmountSubmitted,NetAmount,PayerFees,PayeeFees,FxFees,FxRate,PaymentPlan,PayerId,...,IgnoreTaxValidation,PayerMtState,TaxWithholdingRate,AmountSubmittedUSD,AmountSubmittedUSDUsed,IncomeTypeId,ApplyMaxWithholding,MaxWithholdingReason,EstimatedProviderFeesAmount,EstimatedProviderFeesCurrency
0,445306,292,14932.0,14907.0,0.0,25.0,0.0,0,0,18,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
1,445828,292,14932.0,14907.0,0.0,25.0,0.0,0,0,18,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
2,445253,295,717.0,716.0,0.0,1.0,0.0,0,0,18,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
3,445775,295,717.0,716.0,0.0,1.0,0.0,0,0,18,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
4,445280,415,1921.0,1896.0,0.0,25.0,0.0,0,0,18,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN


In [223]:
dict_counts = df['PayeeId'].value_counts().to_dict()

In [224]:
status_dict = dict()
Id_dict = dict()

for index, row in df.iterrows():
    key = row['PayeeId']
    if key in status_dict:
        status_dict[key].append(row['Status'])
        Id_dict[key].append(row['Id'])
    else:
        status_dict[key] = [row['Status']]
        Id_dict[key] = [row['Id']]

In [225]:
by_id_amount_date = df.groupby(['PayeeId','AmountSubmitted', 'SubmittedDateOnlyDate'])

In [226]:
grouped_status_df = by_id_amount_date['Status'].apply(list)
grouped_id_df = by_id_amount_date['Id'].apply(list)

In [227]:
grouped_status_df.to_csv('grouped_status')
grouped_id_df.to_csv('grouped_id')

In [228]:
grouped_status_df = pd.DataFrame(grouped_status_df)
grouped_status_df2 = grouped_status_df.reset_index()
grouped_status_df2.head()

,PayeeId,AmountSubmitted,SubmittedDateOnlyDate,Status
0,292,14932.0,4/11/2013,"[12, 0]"
1,295,717.0,4/11/2013,"[12, 0]"
2,415,1921.0,4/11/2013,"[12, 0]"
3,620,302.0,8/2/2013,"[10, 0]"
4,620,443.0,4/11/2013,"[12, 0]"


In [229]:
grouped_id_df = pd.DataFrame(grouped_id_df)
grouped_id_df2 = grouped_id_df.reset_index()
new_grouped_id_df = grouped_id_df2[['PayeeId', 'AmountSubmitted', 'SubmittedDateOnlyDate','Id']]
Only_id = new_grouped_id_df['Id']
new_grouped_id_df.head()

,PayeeId,AmountSubmitted,SubmittedDateOnlyDate,Id
0,292,14932.0,4/11/2013,"[445306, 445828]"
1,295,717.0,4/11/2013,"[445253, 445775]"
2,415,1921.0,4/11/2013,"[445280, 445802]"
3,620,302.0,8/2/2013,"[669644, 674587]"
4,620,443.0,4/11/2013,"[445226, 445748]"


In [230]:
grouped_df = pd.concat([grouped_status_df2, Only_id], axis =1)
# grouped_df = pd.append([grouped_status_df2, new_grouped_id_df], axis =1)

selected_grouped_df = grouped_df[['PayeeId','AmountSubmitted','SubmittedDateOnlyDate','Status','Id']]
selected_grouped_df.head()

,PayeeId,AmountSubmitted,SubmittedDateOnlyDate,Status,Id
0,292,14932.0,4/11/2013,"[12, 0]","[445306, 445828]"
1,295,717.0,4/11/2013,"[12, 0]","[445253, 445775]"
2,415,1921.0,4/11/2013,"[12, 0]","[445280, 445802]"
3,620,302.0,8/2/2013,"[10, 0]","[669644, 674587]"
4,620,443.0,4/11/2013,"[12, 0]","[445226, 445748]"


In [231]:

need_check = []

# we are going to check status_list such that
for index, row in selected_grouped_df.iterrows():
# 0: paid, 2: rejected, 5: Pending, 6: deffered, 8: submittedToPaymentSystem, 10: DeferredInternal, 12 = Canceled, 13 = Cleared
    stat = row['Status']
    if all(i==0 or i==8 or i== 13 for i in stat):
        need_check.append(row)
    else:
        continue


In [232]:
new_df = pd.DataFrame.from_records(need_check)

In [233]:
new_df.tail(10)

,PayeeId,AmountSubmitted,SubmittedDateOnlyDate,Status,Id
1368,926864,760.00,7/14/2016,"[0, 0]","[3226837, 3238764]"
1369,931271,90.00,6/29/2016,"[0, 0]","[3179028, 3179032]"
1370,936262,1321.62,7/18/2016,"[0, 0]","[3257093, 3257113]"
1371,941519,500.00,6/27/2016,"[0, 0, 0, 0]","[3158186, 3158187, 3158188, 3158191]"
1372,943570,72.00,6/16/2016,"[0, 0]","[3130150, 3130152]"
1373,946538,225.00,6/20/2016,"[0, 0]","[3146843, 3146844]"
1374,985393,22.50,7/22/2016,"[0, 0]","[3268303, 3268304]"
1375,988676,45.00,7/8/2016,"[0, 0]","[3225805, 3225806]"
1376,1011027,45.00,7/18/2016,"[0, 0, 0]","[3256738, 3256739, 3256740]"
1377,1024418,22.50,7/25/2016,"[0, 0, 0, 0, 0]","[3270390, 3270410, 3270411, 3270412, 3270413]"


In [234]:
new_df.to_csv('grouped_id_status_relevant.csv')

In [235]:
Id_list = new_df['Id'].tolist()

In [236]:
stretched_id_list = []
for l in Id_list:
    for el in l:
        stretched_id_list.append(el)

In [237]:
# stretched_id_list

In [238]:
big_table = []
for el in stretched_id_list:
    row = df.loc[df['Id']== el]
    big_table.append(row)

In [239]:
type(big_table[0])

pandas.core.frame.DataFrame

In [241]:

big_table_df = pd.concat(big_table)
big_table_df.tail()

,Id,PayeeId,AmountSubmitted,NetAmount,PayerFees,PayeeFees,FxFees,FxRate,PaymentPlan,PayerId,...,IgnoreTaxValidation,PayerMtState,TaxWithholdingRate,AmountSubmittedUSD,AmountSubmittedUSDUsed,IncomeTypeId,ApplyMaxWithholding,MaxWithholdingReason,EstimatedProviderFeesAmount,EstimatedProviderFeesCurrency
14242,3270390,1024418,22.5,21.87,0.37,0.63,0.0,0,0,204,...,Indeterminate,CA,NaN,22.5,True,NaN,Indeterminate,NaN,0.4374,USD
14243,3270410,1024418,22.5,21.87,0.37,0.63,0.0,0,0,204,...,Indeterminate,CA,NaN,22.5,True,NaN,Indeterminate,NaN,0.4374,USD
14244,3270411,1024418,22.5,21.87,0.37,0.63,0.0,0,0,204,...,Indeterminate,CA,NaN,22.5,True,NaN,Indeterminate,NaN,0.4374,USD
14245,3270412,1024418,22.5,21.87,0.37,0.63,0.0,0,0,204,...,Indeterminate,CA,NaN,22.5,True,NaN,Indeterminate,NaN,0.4374,USD
14246,3270413,1024418,22.5,21.87,0.37,0.63,0.0,0,0,204,...,Indeterminate,CA,NaN,22.5,True,NaN,Indeterminate,NaN,0.4374,USD


In [248]:
new_id_dict = {}
for index, row in big_table_df.iterrows():
    key = row['PayeeId']
    if key in new_id_dict:
        new_id_dict[key].append(row['Id'])
    else:
        new_id_dict[key] = [row['Id']]

In [256]:
by_id_amount_date2 = big_table_df.groupby(['PayeeId','AmountSubmitted', 'SubmittedDateOnlyDate'])

grouped_pm_df = by_id_amount_date2['PaymentMethod'].apply(list)
grouped_pm_df = pd.DataFrame(grouped_pm_df)

grouped_id_pm_df = by_id_amount_date2['Id'].apply(list)
grouped_id_pm_df = pd.DataFrame(grouped_id_pm_df)

grouped_pm_df2 = grouped_pm_df.reset_index()
grouped_id_pm_df2 = grouped_id_pm_df.reset_index()
#NEEEDS EDIT
pm_with_id = pd.concat([grouped_pm_df2, grouped_id_pm_df2['Id']], axis = 1)
pm_id_df = pm_with_id[['PayeeId','AmountSubmitted','SubmittedDateOnlyDate','PaymentMethod','Id']]



In [257]:
# no_paypal_df = big_table_df[big_table_df.PaymentMethod != 0]
pm_check = []

# we are going to check status_list such that
for index, row in pm_id_df.iterrows():
    payment = row['PaymentMethod']
    try:
        if all(i== 0 for i in payment):
            continue
        else:
            pm_check.append(row)
    except TypeError:
        print(row)

In [258]:
new_df2 = pd.DataFrame.from_records(pm_check)

new_df2.to_csv('grouped_id_pm_relevant.csv')
Id_list2 = new_df2['Id'].tolist()
stretched_id_list2= []

for l in Id_list2:
    for el in l:
        stretched_id_list2.append(el)
        
pm_big_table = []
for el in stretched_id_list2:
    row = df.loc[df['Id']== el]
    pm_big_table.append(row)
    
    
pm_big_table_df = pd.concat(pm_big_table)


,Id,PayeeId,AmountSubmitted,NetAmount,PayerFees,PayeeFees,FxFees,FxRate,PaymentPlan,PayerId,...,IgnoreTaxValidation,PayerMtState,TaxWithholdingRate,AmountSubmittedUSD,AmountSubmittedUSDUsed,IncomeTypeId,ApplyMaxWithholding,MaxWithholdingReason,EstimatedProviderFeesAmount,EstimatedProviderFeesCurrency
383,338934,135994,4126.58,3989.29,-10.0,35.0,102.29,0,0,11,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
384,338935,135994,4126.58,3989.29,-10.0,35.0,102.29,0,0,11,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
626,445189,144215,274.00,274.00,0.0,0.0,0.00,0,0,18,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
627,445711,144215,274.00,274.00,0.0,0.0,0.00,0,0,18,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN
1038,525708,168122,100.00,97.50,5.0,0.0,2.50,0,0,38,...,Indeterminate,NaN,NaN,NaN,False,NaN,Indeterminate,NaN,NaN,NaN


In [260]:
pm_big_table_df.tail()

,Id,PayeeId,AmountSubmitted,NetAmount,PayerFees,PayeeFees,FxFees,FxRate,PaymentPlan,PayerId,...,IgnoreTaxValidation,PayerMtState,TaxWithholdingRate,AmountSubmittedUSD,AmountSubmittedUSDUsed,IncomeTypeId,ApplyMaxWithholding,MaxWithholdingReason,EstimatedProviderFeesAmount,EstimatedProviderFeesCurrency
14228,3146844,946538,225.0,225.00,1.0,0.0,0.00,0,0,204,...,Indeterminate,CA,NaN,225.0,True,NaN,Indeterminate,NaN,NaN,NaN
14233,3268303,985393,22.5,22.05,5.0,0.0,0.56,0,0,204,...,Indeterminate,CA,NaN,22.5,True,NaN,Indeterminate,NaN,NaN,NaN
14234,3268304,985393,22.5,22.05,5.0,0.0,0.56,0,0,204,...,Indeterminate,CA,NaN,22.5,True,NaN,Indeterminate,NaN,NaN,NaN
14235,3225805,988676,45.0,45.00,1.0,0.0,0.00,0,0,204,...,Indeterminate,CA,NaN,45.0,True,NaN,Indeterminate,NaN,NaN,NaN
14236,3225806,988676,45.0,45.00,1.0,0.0,0.00,0,0,204,...,Indeterminate,CA,NaN,45.0,True,NaN,Indeterminate,NaN,NaN,NaN


In [261]:

pm_big_table_df.to_csv('pm_big_table.csv')

In [262]:
no_paypal_df = pm_big_table_df

In [263]:
by_id_am_date = no_paypal_df.groupby(['PayeeId','AmountSubmitted', 'SubmittedDateOnlyDate'])

In [264]:
grouped_m = by_id_am_date['Message'].apply(list)
grouped_mb = by_id_am_date['MessageBanking'].apply(list)


In [265]:
grouped_id2.head()

PayeeId  AmountSubmitted  SubmittedDateOnlyDate
292      14932.0          4/11/2013                [445306, 445828]
295      717.0            4/11/2013                [445253, 445775]
415      1921.0           4/11/2013                [445280, 445802]
620      443.0            4/11/2013                [445226, 445748]
621      389.0            4/11/2013                [445221, 445743]
Name: Id, dtype: object

In [266]:
grouped_m_df = pd.DataFrame(grouped_m)
grouped_m_df2 = grouped_m_df.reset_index()
grouped_m_df2.head()

,PayeeId,AmountSubmitted,SubmittedDateOnlyDate,Message
0,135994,4126.58,12/27/2012,"[nan, nan]"
1,144215,274.00,4/11/2013,"[nan, nan]"
2,168122,100.00,6/18/2013,"[nan, nan]"
3,178054,920.00,1/29/2014,"[Service Provider Seeking Alpha January 2014,..."
4,180483,320.00,1/21/2014,"[nan, nan]"


In [267]:
grouped_mb_df = pd.DataFrame(grouped_mb)
grouped_mb_df2 = grouped_mb_df.reset_index()
# new_grouped_id_df = grouped_id_df2[['PayeeId', 'AmountSubmitted', 'SubmittedDateOnlyDate','Id']]
# Only_id = new_grouped_id_df['Id']
# new_grouped_id_df.head()
grouped_mb_df2.head()

,PayeeId,AmountSubmitted,SubmittedDateOnlyDate,MessageBanking
0,135994,4126.58,12/27/2012,"[nan, nan]"
1,144215,274.00,4/11/2013,"[nan, nan]"
2,168122,100.00,6/18/2013,"[nan, nan]"
3,178054,920.00,1/29/2014,"[nan, nan]"
4,180483,320.00,1/21/2014,"[TouchOfModern PO0068179605 / 040, TouchOfMode..."


In [268]:
grouped_id2 = by_id_am_date['Id'].apply(list)
grouped_id2_df = pd.DataFrame(grouped_id2)
grouped_id2_df2 = grouped_id2_df.reset_index()

In [269]:
new_grouped_id_df2 = grouped_id2_df2[['PayeeId', 'AmountSubmitted', 'SubmittedDateOnlyDate','Id']]
Only_id2 = new_grouped_id_df2['Id']
Only_id2.tail()

969    [3158186, 3158187, 3158188, 3158191]
970                      [3130150, 3130152]
971                      [3146843, 3146844]
972                      [3268303, 3268304]
973                      [3225805, 3225806]
Name: Id, dtype: object

In [270]:
grouped_df1 = pd.concat([grouped_m_df2, Only_id2], axis =1)
grouped_df2 = pd.concat([grouped_mb_df2, Only_id2], axis =1)
# grouped_df = pd.append([grouped_status_df2, new_grouped_id_df], axis =1)

selected_grouped_df2 = grouped_df1[['PayeeId','AmountSubmitted','SubmittedDateOnlyDate','Message','Id']]
selected_grouped_df3 = grouped_df2 [['PayeeId','AmountSubmitted','SubmittedDateOnlyDate','MessageBanking','Id']]


In [271]:
selected_combined = pd.concat([selected_grouped_df2, selected_grouped_df3['MessageBanking']], axis = 1)
selected_combined.tail()

selected_combined.to_csv('selected_combined.csv')

In [272]:
message_check = []

def all_same(items):
    if [x for x in items if str(x) == 'nan'] != []:
        return len(items) == len([x for x in items if str(x) == 'nan'] )
    else:
        return all(x == items[0] for x in items)


for index, row in selected_combined.iterrows():
    m = row['Message']
    mb = row['MessageBanking']
    if all_same(m) and all_same(mb):
        message_check.append(row)
    else:
        continue

In [273]:
new_m_mb_df = pd.DataFrame.from_records(message_check)


In [274]:
new_m_mb_df.tail()

,PayeeId,AmountSubmitted,SubmittedDateOnlyDate,Message,Id,MessageBanking
388,891103,79.20,7/1/2016,"[nan, nan]","[3129163, 3154117]","[nan, nan]"
389,916191,274.50,5/30/2016,"[nan, nan]","[3075971, 3076025]","[nan, nan]"
390,926864,760.00,7/14/2016,"[nan, nan]","[3226837, 3238764]","[nan, nan]"
391,936262,1321.62,7/18/2016,"[nan, nan]","[3257093, 3257113]","[Payout for event 25907458907., Payout for eve..."
392,941519,500.00,6/27/2016,"[nan, nan, nan, nan]","[3158186, 3158187, 3158188, 3158191]","[nan, nan, nan, nan]"


In [275]:
new_m_mb_df.to_csv("new_m_mb_df.csv")

In [276]:
Id_list3 = new_m_mb_df['Id'].tolist()
stretched_id_list3= []

for l in Id_list3:
    for el in l:
        stretched_id_list3.append(el)
        
paymentmethod_big_table = []
for el in stretched_id_list3:
    row = df.loc[df['Id']== el]
    paymentmethod_big_table.append(row)
    
    
pm_big_table_df2 = pd.concat(paymentmethod_big_table)


In [277]:
pm_big_table_df2.tail()

,Id,PayeeId,AmountSubmitted,NetAmount,PayerFees,PayeeFees,FxFees,FxRate,PaymentPlan,PayerId,...,IgnoreTaxValidation,PayerMtState,TaxWithholdingRate,AmountSubmittedUSD,AmountSubmittedUSDUsed,IncomeTypeId,ApplyMaxWithholding,MaxWithholdingReason,EstimatedProviderFeesAmount,EstimatedProviderFeesCurrency
14220,3257113,936262,1321.62,1321.62,16.39,0.0,0.0,0,0,157,...,Indeterminate,NaN,NaN,403.093,True,NaN,Indeterminate,NaN,NaN,NaN
14221,3158186,941519,500.00,500.00,1.00,0.0,0.0,0,0,167,...,Indeterminate,CA,NaN,500.000,True,8.0,Indeterminate,NaN,NaN,NaN
14222,3158187,941519,500.00,500.00,1.00,0.0,0.0,0,0,167,...,Indeterminate,CA,NaN,500.000,True,8.0,Indeterminate,NaN,NaN,NaN
14223,3158188,941519,500.00,500.00,1.00,0.0,0.0,0,0,167,...,Indeterminate,CA,NaN,500.000,True,8.0,Indeterminate,NaN,NaN,NaN
14224,3158191,941519,500.00,500.00,1.00,0.0,0.0,0,0,167,...,Indeterminate,CA,NaN,500.000,True,8.0,Indeterminate,NaN,NaN,NaN


In [278]:
pm_big_table_df2.to_csv('including_payment_method_df.csv')